imort mxnet as mx


In [ ]:
import mxnet as mx
from mxnet import nd
from mxnet import gluon
from mxnet.gluon import nn

class MLP(nn.Block):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        with self.name_scope():
            self.dense0 = nn.Dense(256)
            self.dense1 = nn.Dense(10)
        
    def forward(self, x):
        return self.dense1(nd.relu(self.dense0(x)))
    

In [2]:
# 查看
net2 = MLP()
print(net2)

net2.initialize()
print(net2)

x = nd.random.uniform(shape=(4,20))
y = net2(x)

print('*' * 30)
print('x.shape is: ', x.shape)
print('y.shape is: ', y.shape)

MLP(
  (dense0): Dense(None -> 256, linear)
  (dense1): Dense(None -> 10, linear)
)
MLP(
  (dense0): Dense(None -> 256, linear)
  (dense1): Dense(None -> 10, linear)
)
******************************
x.shape is:  (4, 20)
y.shape is:  (4, 10)


In [3]:
print('default name: ', net2.name)
print('default prefix: ', net2.prefix)
print('default name of net2.dense0: ', net2.dense0.name)
print('default prefix of net2.dense0: ', net2.dense0.prefix)

default name:  mlp0
default prefix:  mlp0_
default name of net2.dense0:  mlp0_dense0
default prefix of net2.dense0:  mlp0_dense0_


In [4]:
# gluon.nn.Sequential的简单含义：Block的子类，存储子Block对象的容器
class Sequential(nn.Block):
    def __init__(self, **kwargs):  # 传入实参应为字典或键值对
        super(Sequential, self).__init__(**kwargs)  # 调用父类构造方法
    def add(self, block):
        #pass
        self._children.append(block)  # 将block添加入成员变量（list) ***error: 运行提示OrderedDict没有append方法
    def forward(self, x):
        for block in _children:
            x = block(x)
        return x    

In [5]:
net4 = Sequential()
with net4.name_scope():
    net4.add(gluon.nn.Dense(256, activation='relu'))
    net4.add(gluon.nn.Dense(10))
net4.initialize()

AttributeError: 'collections.OrderedDict' object has no attribute 'append'

In [ ]:
class FancyMLP(gluon.nn.Block):
    def __init__(self, **kwargs):
        super(FancyMLP,self).__init__(**kwargs)
        with self.name_scope():
            self.dense = gluon.nn.Dense(256)
            self.weight = nd.random_uniform(shape=(256, 20))
    def forward(self, x):
        x = nd.relu(self.dense(x))
        x = nd.relu(nd.dot(x, self.weight) + 1)
        x = nd.relu(self.dense(x))
        return x

In [6]:
f = FancyMLP()
f.initialize()
data = random

NameError: name 'FancyMLP' is not defined

In [7]:
f.dense(x)

NameError: name 'f' is not defined

In [ ]:
# Block和Sequential嵌套 示例
class RecMLP(gluon.nn.Block):  # recurisive嵌套
    def __init__(self, **kwargs):
        super(RecMLP, self).__init__(**kwargs)
        self.net = nn.Sequential()
        with self.name_scope():
            self.net.add(gluon.nn.Dense(256, activation='relu'))
            self.net.add(gluon.nn.Dense(128, activation='relu'))
            self.dense = gluon.nn.Dense(64)
    def forward(self, x):
        return nd.relu(self.dense(self.net(x)))


In [9]:
# initialize的意义
# 获取模型
def get_net():
    net = gluon.nn.Sequential()
    with net.name_scope():
        net.add(gluon.nn.Dense(4, activation='relu'))
        net.add(gluon.nn.Dense(2))
    return net

In [10]:
# 不initialize直接跑
import sys
try:
    net = get_net()
    net(x)
except RuntimeError as RT_err:
    sys.stderr.write(str(RT_err))

Parameter 'sequential1_dense0_weight' has not been initialized. Note that you should initialize parameters and create Trainer with Block.collect_params() instead of Block.params because the later does not include Parameters of nested child Blocks

In [11]:
net.initialize()
net(x)


[[-0.00185824  0.00324628]
 [-0.00263331  0.00166448]
 [-0.00444654  0.00165451]
 [ 0.00041254  0.0011421 ]]
<NDArray 4x2 @cpu(0)>

In [12]:
net[0].params

sequential1_dense0_ (
  Parameter sequential1_dense0_weight (shape=(4, 20), dtype=float32)
  Parameter sequential1_dense0_bias (shape=(4,), dtype=float32)
)

In [13]:
net.collect_params()

sequential1_ (
  Parameter sequential1_dense0_weight (shape=(4, 20), dtype=float32)
  Parameter sequential1_dense0_bias (shape=(4,), dtype=float32)
  Parameter sequential1_dense1_weight (shape=(2, 4), dtype=float32)
  Parameter sequential1_dense1_bias (shape=(2,), dtype=float32)
)

In [14]:
net

Sequential(
  (0): Dense(20 -> 4, Activation(relu))
  (1): Dense(4 -> 2, linear)
)

In [15]:
net

Sequential(
  (0): Dense(20 -> 4, Activation(relu))
  (1): Dense(4 -> 2, linear)
)

In [16]:
net[-1]

Dense(4 -> 2, linear)

In [17]:
net[-2]

Dense(20 -> 4, Activation(relu))

In [18]:
net1 = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Dense(8, in_units=4, activation='relu'))
    net.add(gluon.nn.Dense(4, in_units=4, activation='relu'))
    net.add(gluon.nn.Dense(8, activation='relu', params=net[0].params))
    net.add(gluon.nn.Dense(2))


AssertionError: Cannot retrieve Parameter 'sequential1_dense0_weight' because desired attribute does not match with stored for attribute 'shape': desired '(8, 0)' vs stored '(4, 20)'.

In [19]:
net.collect_params()

sequential1_ (
  Parameter sequential1_dense0_weight (shape=(4, 20), dtype=float32)
  Parameter sequential1_dense0_bias (shape=(4,), dtype=float32)
  Parameter sequential1_dense1_weight (shape=(2, 4), dtype=float32)
  Parameter sequential1_dense1_bias (shape=(2,), dtype=float32)
  Parameter sequential1_dense2_weight (shape=(8, 4), dtype=float32)
  Parameter sequential1_dense2_bias (shape=(8,), dtype=float32)
  Parameter sequential1_dense3_weight (shape=(4, 4), dtype=float32)
  Parameter sequential1_dense3_bias (shape=(4,), dtype=float32)
)

In [20]:
net[2].weight.data()

RuntimeError: Parameter 'sequential1_dense2_weight' has not been initialized. Note that you should initialize parameters and create Trainer with Block.collect_params() instead of Block.params because the later does not include Parameters of nested child Blocks

In [21]:
x = nd.rando

AttributeError: module 'mxnet.ndarray' has no attribute 'rando'

In [22]:
x


[[5.4881352e-01 5.9284461e-01 7.1518934e-01 8.4426576e-01 6.0276335e-01
  8.5794562e-01 5.4488319e-01 8.4725171e-01 4.2365479e-01 6.2356371e-01
  6.4589411e-01 3.8438171e-01 4.3758720e-01 2.9753461e-01 8.9177299e-01
  5.6712978e-02 9.6366274e-01 2.7265629e-01 3.8344151e-01 4.7766513e-01]
 [7.9172504e-01 8.1216872e-01 5.2889490e-01 4.7997716e-01 5.6804454e-01
  3.9278480e-01 9.2559665e-01 8.3607876e-01 7.1036056e-02 3.3739617e-01
  8.7129295e-02 6.4817190e-01 2.0218398e-02 3.6824155e-01 8.3261985e-01
  9.5715517e-01 7.7815676e-01 1.4035077e-01 8.7001216e-01 8.7008727e-01]
 [4.1702199e-01 9.9718481e-01 7.2032452e-01 9.3255734e-01 1.1438108e-04
  1.2812445e-01 3.0233258e-01 9.9904054e-01 1.4675589e-01 2.3608898e-01
  9.2338592e-02 3.9658073e-01 1.8626021e-01 3.8791075e-01 3.4556073e-01
  6.6974604e-01 3.9676747e-01 9.3553907e-01 5.3881675e-01 8.4631091e-01]
 [4.1919452e-01 3.1327352e-01 6.8521953e-01 5.2454817e-01 2.0445225e-01
  4.4345289e-01 8.7811744e-01 2.2957721e-01 2.7387597e-02 5.

In [23]:
net

Sequential(
  (0): Dense(20 -> 4, Activation(relu))
  (1): Dense(4 -> 2, linear)
  (2): Dense(4 -> 8, Activation(relu))
  (3): Dense(4 -> 4, Activation(relu))
)

In [24]:
net(x)

RuntimeError: Parameter 'sequential1_dense2_weight' has not been initialized. Note that you should initialize parameters and create Trainer with Block.collect_params() instead of Block.params because the later does not include Parameters of nested child Blocks

In [25]:
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Dense(16,  activation='relu'))
    net.add(gluon.nn.Dense(8,  activation='relu'))
    net.add(gluon.nn.Dense(2))

In [26]:
net.initialize()

In [27]:
net(x)


[[-0.00202598 -0.00073244]
 [-0.00298761 -0.00085276]
 [-0.0022157  -0.00120266]
 [-0.00104701 -0.00085004]]
<NDArray 4x2 @cpu(0)>

In [28]:
file_name = '/home/frank/MXNet_Learning/Lesson_3/nd_save_demon.params'

In [29]:
nd.save(file_name, x)
nd.save('x.txt', x)

In [30]:
a = nd.load(file_name)
b = nd.load('x.txt')

In [31]:
net.save_params(filename='net_save_demon.params')

In [32]:
model = get_net()

In [33]:
# 自定义一个简单特定功能的层：功能：输入-均值；缺点：不具有优化参数
# 自己的理解：只需要override构造方法和forward方法即可
# 实例化后：layer(x)自动绑定forward方法
class CenteredLayer(gluon.nn.Block):
    def __init__(self, **kwargs):
        super(CenteredLayer, self).__init__(**kwargs)
    def forward(self, x):
        return x - x.mean()

In [34]:
m = gluon.nn.Dense(1)

In [35]:
m.initialize()

In [36]:
layer = CenteredLayer()


In [37]:
layer(x)


[[ 0.02171755  0.06574863  0.18809336  0.3171698   0.07566738  0.33084965
   0.01778722  0.32015574 -0.10344118  0.09646773  0.11879814 -0.14271426
  -0.08950877 -0.22956136  0.364677   -0.470383    0.43656677 -0.25443968
  -0.14365447 -0.04943085]
 [ 0.26462907  0.28507274  0.00179893 -0.04711881  0.04094857 -0.13431117
   0.39850068  0.3089828  -0.45605993 -0.1896998  -0.43996668  0.12107593
  -0.5068776  -0.15885442  0.30552387  0.4300592   0.25106078 -0.3867452
   0.3429162   0.3429913 ]
 [-0.11007398  0.47008884  0.19322854  0.40546137 -0.5269816  -0.39897153
  -0.2247634   0.47194457 -0.3803401  -0.29100698 -0.43475738 -0.13051525
  -0.34083575 -0.13918522 -0.18153524  0.14265007 -0.1303285   0.4084431
   0.01172078  0.31921494]
 [-0.10790145 -0.21382245  0.15812355 -0.0025478  -0.32264373 -0.08364308
   0.35102147 -0.29751876 -0.49970838  0.00731796  0.14337152  0.38686603
  -0.10979116 -0.06989115  0.03159386 -0.0963974  -0.38670903  0.41203183
  -0.32899448  0.25129324]]
<NDA

In [38]:
layer.forward(x)


[[ 0.02171755  0.06574863  0.18809336  0.3171698   0.07566738  0.33084965
   0.01778722  0.32015574 -0.10344118  0.09646773  0.11879814 -0.14271426
  -0.08950877 -0.22956136  0.364677   -0.470383    0.43656677 -0.25443968
  -0.14365447 -0.04943085]
 [ 0.26462907  0.28507274  0.00179893 -0.04711881  0.04094857 -0.13431117
   0.39850068  0.3089828  -0.45605993 -0.1896998  -0.43996668  0.12107593
  -0.5068776  -0.15885442  0.30552387  0.4300592   0.25106078 -0.3867452
   0.3429162   0.3429913 ]
 [-0.11007398  0.47008884  0.19322854  0.40546137 -0.5269816  -0.39897153
  -0.2247634   0.47194457 -0.3803401  -0.29100698 -0.43475738 -0.13051525
  -0.34083575 -0.13918522 -0.18153524  0.14265007 -0.1303285   0.4084431
   0.01172078  0.31921494]
 [-0.10790145 -0.21382245  0.15812355 -0.0025478  -0.32264373 -0.08364308
   0.35102147 -0.29751876 -0.49970838  0.00731796  0.14337152  0.38686603
  -0.10979116 -0.06989115  0.03159386 -0.0963974  -0.38670903  0.41203183
  -0.32899448  0.25129324]]
<NDA

In [39]:
# 自定义带优化参数的层：
class MyDense(gluon.nn.Block):
    def __init__(self, units, in_units, **kwargs):
        super(MyDense, self).__init__(units, in_units, **kwargs)
        with self.name_scope():
            self.weight = self.params.get('weight', shape=(in_units, units))
            self.bias = self.params.get('bias', shape=(units,))
    def forward(self, x):
        linear = nd.dot(x, self.weight.data()) + self.bias.data()
        return nd.relu(linear)

In [40]:
# dropout正则化方法
def dropout(X, drop_prob):
    keep_prob = 1 - drop_prob
    assert 0 <= keep_prob <= 1, 'prob must be in (0, 1)'
    if keep_prob == 0:
        return X.zeros_like()
    # mask 用以屏蔽某条件的元素
    mask = nd.random.uniform(0,1, shape=X.shape) < keep_prob
    # scale用于保证E(X)不变，drop掉某些点后，E(X)后减小
    scale = 1 / keep_prob
    return X * mask * scale   

In [41]:
dropout(x, 0.5)


[[1.0976270e+00 0.0000000e+00 0.0000000e+00 1.6885315e+00 0.0000000e+00
  0.0000000e+00 1.0897664e+00 0.0000000e+00 0.0000000e+00 1.2471274e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 7.6688302e-01 0.0000000e+00]
 [1.5834501e+00 1.6243374e+00 1.0577898e+00 9.5995432e-01 1.1360891e+00
  7.8556961e-01 0.0000000e+00 0.0000000e+00 0.0000000e+00 6.7479235e-01
  1.7425859e-01 0.0000000e+00 4.0436797e-02 0.0000000e+00 1.6652397e+00
  1.9143103e+00 1.5563135e+00 2.8070155e-01 1.7400243e+00 1.7401745e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 1.8651147e+00 2.2876216e-04
  0.0000000e+00 0.0000000e+00 1.9980811e+00 0.0000000e+00 4.7217795e-01
  1.8467718e-01 0.0000000e+00 0.0000000e+00 7.7582151e-01 0.0000000e+00
  0.0000000e+00 0.0000000e+00 1.8710781e+00 1.0776335e+00 1.6926218e+00]
 [8.3838904e-01 0.0000000e+00 1.3704391e+00 1.0490963e+00 0.0000000e+00
  8.8690579e-01 1.7562349e+00 4.5915443e-01 0.0000000e+00 1.

In [48]:
nd.random_uniform(0,1,shape=(1,6)) < 0.5


[[1. 1. 0. 0. 1. 0.]]
<NDArray 1x6 @cpu(0)>

In [53]:
nd.random.uniform(0,1,shape=(1,6)) < 0.5


[[0. 1. 1. 0. 0. 1.]]
<NDArray 1x6 @cpu(0)>